In [ ]:
print("ok")

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
PINECONE_API_KEY = "2c8650bc-84f5-4999-8b8d-045d8ef905aa"

In [ ]:
# EXTRACT DATA FROM PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [ ]:
extracted_data = load_pdf(data="data/")

In [ ]:
# create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of the chunk:",len(text_chunks))

In [ ]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
embeddings

In [ ]:
# query_result = embeddings.embed_query("hELLO tANMAY")
# print(len(query_result))


In [ ]:
import os

In [ ]:
# from langchain_pinecone import PineconeVectorStore
os.environ['PINECONE_API_KEY'] = '2c8650bc-84f5-4999-8b8d-045d8ef905aa'
index_name = 'medical-chatbot'

In [ ]:
# os.environ['PINECONE_API_KEY'] = '2c8650bc-84f5-4999-8b8d-045d8ef905aa'



# docsearch = PineconeVectorStore.from_texts(
#     [t.page_content for t in text_chunks],
#     embeddings,
#     index_name = index_name
# )

In [ ]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)

query = "medicine for cold"

docs = docsearch.similarity_search(query, k=3)
print("Result:", docs)

In [ ]:
prompt_template = """
Use the following pieces of infoormation to answer the user and if you do not know the answer to user's query, jut say you do not know, please do not make up the answer.

Context: {context}
Question: {question}

Only return helpful answer nothing else.
Helpful answer:
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
chain_type_kwargs={"prompt": PROMPT}

In [ ]:
llm = CTransformers(model="model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens': 512,
                            'temperature':0.8})

In [ ]:
qa_retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={'k':2},
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=qa_retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)


In [ ]:
while True:
    user_input=input(f"Input Prompt: ")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

In [ ]:
# /end 1

In [ ]:
# class PineconeConnected():
#     def __init__(self, index_name: str, pinecone_api_key: str):
#         embeddings = embeddings
#         self.pinecone = pinecone.Pinecone(api_key=PINECONE_API_KEY)
#         self.vector_db = Pinecone.from_existing_index("medical=chatbot", embeddings) # VectorStore object with the reference + Pinecone index loaded

#     def query(query:str, book_title=None):
        
#         pass

In [ ]:
# # pinecone initialized
# pinecone.init(api_key=PINECONE_API_KEY)

# index_name = "medical=chatbot"

# # Creating embeddings for each of the text chunks and storing
# docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],
#                                 embeddings, index_name=index_name)

In [ ]:
# # Pinecone initialization (correct API key provided)
# import os
# from pinecone import Pinecone, ServerlessSpec

# YOUR_PINECONE_API_KEY = "2c8650bc-84f5-4999-8b8d-045d8ef905aa"  # Replace with your real API key
# pc = Pinecone(api_key=YOUR_PINECONE_API_KEY)

# index_name = "medical=chatbot"

# # Creating embeddings for each of the text chunks and storing
# docsearch = pc.from_texts([t.page_content for t in text_chunks],
#                           embeddings, index_name=index_name)
